You can replace `submission_v1.csv` with your own public test preds 

In [ ]:
from fastai.vision.all import *

In [ ]:
datapath = Path("/kaggle/input/hubmap-kidney-segmentation/")

In [ ]:
sample_subdf = pd.read_csv(datapath/"sample_submission.csv")
subdf = pd.read_csv("/kaggle/input/hubmapsubmissions/submission_v1.csv")

In [ ]:
subdf

In [ ]:
id2rle = dict(zip(subdf['id'], subdf['predicted']))

### rle utils

In [ ]:
def enc2mask(encs, shape):
    "shape : (w,h)"
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

### Filter out struct

In [ ]:
import rasterio, cv2, gc

In [ ]:
def get_struct_mask(dataset, structs, names=None):
    "Generate mask for anatomical struct including only names"
    struct_mask = np.zeros(dataset.shape)

    for _,row in structs.iterrows():
        if names is None:
            coords = array(row['geometry']['coordinates'][0])
            struct_mask = cv2.fillPoly(struct_mask, pts =[coords], color=255);
        
        elif row['properties']['classification']['name'] in names:   
            coords = array(row['geometry']['coordinates'][0])
            struct_mask = cv2.fillPoly(struct_mask, pts =[coords], color=255);

    return struct_mask.astype(np.uint8)

In [ ]:
# def get_cortex_mask(dataset, structs):
#     cortex_mask = np.zeros(dataset.shape)

#     for _,row in structs.iterrows():
#         if row['properties']['classification']['name'] == 'Cortex':   
#             coords = array(row['geometry']['coordinates'][0])
#             cortex_mask = cv2.fillPoly(cortex_mask, pts =[coords], color=255);

#     return cortex_mask.astype(np.uint8)

In [ ]:
public_testids = ['afa5e8098', 'b9a3865fc', 'c68fe75ea', 'b2dc8411c', '26dc41664']

In [ ]:
testid = public_testids[0]

In [ ]:
new_id2rle = {}

In [ ]:
for testid in progress_bar(public_testids):

    structs = pd.read_json(datapath/f'test/{testid}-anatomical-structure.json')

    dataset = rasterio.open(datapath/f'test/{testid}.tiff') 


#     cortex_mask = get_cortex_mask(dataset, structs)
    cortex_mask = get_struct_mask(dataset, structs)

    rle_mask = enc2mask([id2rle[testid]], dataset.shape[::-1])

    # set non-cortex preds to 0
    rle_mask[cortex_mask == 0] = 0

    del cortex_mask, dataset
    gc.collect()

    new_rle = rle_encode_less_memory(rle_mask)
    
    new_id2rle[testid] = new_rle

In [ ]:
subdf['predicted'] = subdf['id'].map(new_id2rle)

### Submit

In [ ]:
sample_subdf = sample_subdf[['id']].merge(subdf, on='id', how='left')
sample_subdf['predicted'] = sample_subdf['predicted'].fillna('1 1')

In [ ]:
sample_subdf.to_csv("submission.csv",index=False)

In [ ]:
sample_subdf